In [ ]:
from phonecodes import phonecodes
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from timeit import default_timer as timer
from torch.nn import Transformer
from torch import Tensor
from sklearn.model_selection import train_test_split
import copy
import tqdm
import librosa
import seaborn as sns
import torch.nn as nn
import torch
import torch.nn.functional as F
import numpy as np
import math
import os
import pandas as pd
import matplotlib.pyplot as plt
import textgrid
from scipy.spatial.distance import euclidean
import plotly.graph_objects as go
import pandas as pd
import jiwer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import nltk
nltk.download('cmudict')
from nltk.corpus import cmudict
from transformers import AutoProcessor, AutoModelForCTC
from phonemizer.backend.espeak.wrapper import EspeakWrapper
import soundfile as sf
import pickle
_ESPEAK_LIBRARY = r"C:\Program Files\eSpeak NG\libespeak-ng.dll"
EspeakWrapper.set_library(_ESPEAK_LIBRARY)
'''processor_P = AutoProcessor.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")
model_P = AutoModelForCTC.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")'''

# loading Hubert model
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
processor_H = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")
model_H = AutoModelForCTC.from_pretrained("facebook/hubert-large-ls960-ft")

In [ ]:
import random
predicted_df=pd.read_csv("predicted_study1_TS_max.csv")
df=copy.deepcopy(predicted_df)
df["Keyword"]=df["SentenceID"].astype(str)+"_"+df["Keyword"].astype(str)
test_talker_groups = df.groupby("TestTalkerID")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()
test_talkerID=["CMN_M_032","CMN_M_043","CMN_M_035","CMN_M_037"]
condition_order = df["Condition2"].unique() 
palette = sns.color_palette("muted", len(condition_order))  
palette=["gray","green","blue","red"]
for i, (test_talker_id, group) in enumerate(test_talker_groups):
    ax = axes[i]

    group = group.sort_values(by=["Condition2", "TrainingTalkerID1"], key=lambda x: x.map({k: i for i, k in enumerate(condition_order)}))

    training_talker_order = group["TrainingTalkerID1"].unique()

    '''sns.stripplot(
        data=group,
        x="TrainingTalkerID",
        y="similarity",
        hue="Condition2",
        dodge=False,  
        ax=ax,
        palette=palette,
        hue_order=condition_order,
        order=training_talker_order, 
        alpha=0.3,  
        size=5,     
        jitter=True 
    )'''
    
    keyword_sample=random.sample(list(set(list(group["Keyword"]))),15)
    filtered_group= group[group["Keyword"].isin(keyword_sample)]
    print(keyword_sample)
    
    from scipy import stats
    #means = group.groupby("TrainingTalkerID")['similarity'].mean()
    #cis = group.groupby("TrainingTalkerID")['similarity'].apply(lambda x: stats.t.interval(0.95, len(x)-1, loc=np.mean(x), scale=stats.sem(x)))
    sns.lineplot(
        data=filtered_group,
        ax=ax,
        x="TrainingTalkerID1",
        y='similarity',
        units="Keyword",
        estimator=None,
        color="gray",
        alpha=1,
        linewidth=0.4
    )
    
    sns.violinplot(
        data=group,
        x="TrainingTalkerID1",
        y="similarity",
        hue="Condition2",
        # split=True,  
        inner='point',  
        ax=ax,
        alpha=0.5,
        cut=0,
        palette=palette,
        hue_order=condition_order,
        order=training_talker_order,
        linewidth=0
    )
    sns.pointplot(
        data=group,
        x="TrainingTalkerID1",
        y="similarity",
        hue="Condition2",
        ax=ax,
        palette=palette,
        hue_order=condition_order,
        order=training_talker_order,
        linestyles='none',
        estimator=np.mean, 
        #join=False, 
        markers="_",
        markeredgewidth=2,
        markersize=20
        #errwidth=1.5,
        
    )


    ax.legend_.remove()
    ax.set_ylim(0,1.05)
    ax.set_title(f"Sample test talker: {test_talkerID[test_talker_id-3]}")
    ax.set_ylabel("Exposure-to-test talker-similarity of keyword")
    ax.set_xlabel("Exposure Talker(s)")
    ax.set_xticklabels([])  

handles, labels = axes[0].get_legend_handles_labels()
fig.legend(
    handles[:len(condition_order)],  
    labels[:len(condition_order)],
    loc='upper center',
    title="",
    ncol=4,  
    frameon=False,
    fontsize=10
)

plt.tight_layout(rect=[0, 0, 1, 0.95])  
plt.show()
